### Devoir : scrapping de l'ensemble des produits sur opendata.org.
https://fr.openfoodfacts.org/

### Présenté par : 
| Prénoms       |     Nom         |   
| ------------- |: -------------: |
| Amadou lamarana      | DIALLO               |


### Procédures de récupération des données
1. Récupérer les urls qui pointe sur toutes ces pages i.e (https://fr.openfoodfacts.org/3) la 3ieme page
2. Récuperer les urls des produits contenu dans chacune de ces pages
3. Récupération des balises qui contiennent l'information pertinente (mentionnée ci dessous) et création d'une liste avec l'ensemble de toutes ces informations par start-up
4. Création du Dataframe et l'export sous CSV

In [9]:
## Opendata
## import des modules nécessaires
import pandas as pd 
import numpy as np
import random
import time
import re
import requests
from bs4 import BeautifulSoup


    * liens principaux:
        * BASE_URL = https://fr.openfoodfacts.org : page principale

In [26]:
BASE_URL = "https://fr.openfoodfacts.org"

#### Etape 1 : Récupérer les urls qui pointe sur toutes ces pages 

In [62]:
def recuperer_urls_page():
    
    import re
    """
        * Cette fonction retourne la liste des pages du site 
        * Cette liste initialisé avec l'url de valeur BASE_URL
        * Une requête html avec ** request.get(BASE_URL)** pour tenter de récuperer le contenu de l'accueil
        * Si la page est accessible, une série de manipulation avec BeautifoulSoup sera déclenchée pour 
        * suivant le format de liens, BASE_URL/n avec n le numéro de page, nous allons itérer 
            en tenant compte du numéro de la dernière page disponible sur le site.
    @return list: urls
    """
    #Initiate urls 
    urls= [BASE_URL] ## url de base initialisé avec le lien d'accueil du site (lien de base)
    accueil = requests.get(BASE_URL) ## request html 
    if accueil.status_code == requests.codes.ok: ## All is OK (codeResponse = 200)
        ## parser html
        bs = BeautifulSoup(accueil.text, 'html.parser') 
        ## la pagination est accessible par la balise 'ul' et par l'identifiant "pages"
        ulPages = bs.find('ul',id='pages')
        ## récupérer les éléments de la pagination avec la balise 'li'
        page = ulPages.findAll('li', class_='')
        ## extrait avec le module :'re' des numéros de pages qui seront stockés dans une liste "numeros"
        numeros = [ int(n.text) for n in page if n.text in re.findall('[0-9]*', n.text)]
        derniere_page = max(numeros)
        for i in range(2,derniere_page+1):
            url = BASE_URL+'/'+str(i)
            urls.append(url)
    return urls
#print(recuperer_urls_page())

#### Etape 2:  Récuperer les urls des produits contenu dans chacune de ces pages

##### Etape 2.1:  Récuperer les urls des produits contenu dans une page

In [77]:
def recuperer_url_produits_page(url_page):
    """Focntion qui renvoie les urls des produits listé dans la page du lien passé en paramètre"""
    page = requests.get(url_page)
    if page.status_code == requests.codes.ok:
        bs = BeautifulSoup(page.text, 'html.parser')
        bloc_produits = bs.find('ul', class_ ='products')
        #print(bs)
        les_produits = bloc_produits.findAll('li',class_='')
        urls_produits = [BASE_URL+url.find("a")['href'] for url in les_produits]
    return urls_produits

  #####  ETAPE 2. 2  Recupérer les urls de tous des produits de toute les pages 

In [ ]:

def recuperer_urls_produits_site(urls_pages):
    """Focntion qui récupére les liens des produits du site
        * Nous utilisons la fonction "recuperer_urls_page" pour avoir les liens des pages;
        * Nous utilisons la fonction "recuperer_url_produits_page"  pour récupérer 
        les liens des produits par page
        @return list: urls_produits
    """
    urls_produits =[]
    ## les produits par page
    for page in urls_pages:
        #time.sleep(random.uniform(0.2,0.5))
        lien_produits = recuperer_url_produits_page(page)
        
        urls_produits = urls_produits + lien_produits
    return urls_produits

In [94]:
pages = recuperer_urls_page()


In [11]:
## recuperer les urls des produits, enregistrer à la variable urls_produits
urls_produits_site = recuperer_urls_produits_site(pages)


In [138]:
#urls_produits_site = url_0_1000 + url_1000_2000 + url_2000_3000+ url_3000_4000+url_4000_5000 +url_5000_6000+url_6000_7000+url_7000_fin

In [139]:
len(urls_produits_site)

798000

In [146]:
df_url = pd.DataFrame(urls_produits_site)
df_url.columns=['url_product']

In [148]:
df_url.to_csv('url_product',index=False , encoding='utf-8')

In [10]:
df_urls_produits_site = pd.read_csv('url_product.csv')
urls_produits = list(df_urls_produits_site['url_product'])

####  Etape 3 : Recupération des informations des produits 